In [1]:
!pip install transformers datasets scikit-learn

In [2]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments

In [3]:
from google.colab import files
uploaded = files.upload()

Saving train.csv to train.csv


In [4]:
!pip install chardet

# Import the chardet module
import chardet

# Detect the encoding of the CSV file
with open('/content/train.csv', 'rb') as f:
    encoding = chardet.detect(f.read())['encoding']

# Read the CSV file with the detected encoding
df = pd.read_csv('/content/train.csv', encoding=encoding)

# Verify that the data is loaded correctly

In [5]:
df = df[['text', 'sentiment']]
df = df.dropna()
df.head()

,text,sentiment
0,"I`d have responded, if I were going",neutral
1,Sooo SAD I will miss you here in San Diego!!!,negative
2,my boss is bullying me...,negative
3,what interview! leave me alone,negative
4,"Sons of ****, why couldn`t they put them on t...",negative


In [6]:
le = LabelEncoder()
df['sentiment'] = le.fit_transform(df['sentiment'])
label_names = le.classes_
print("Label classes:", list(label_names))

Label classes: ['negative', 'neutral', 'positive']


In [7]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['text'].tolist(), df['sentiment'].tolist(), test_size=0.2, random_state=42
)

In [8]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [9]:
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, sentiment):
        self.encodings = encodings
        self.sentiment = sentiment

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.sentiment[idx])
        return item

    def __len__(self):
        return len(self.sentiment)

train_dataset = SentimentDataset(train_encodings, train_labels)
val_dataset = SentimentDataset(val_encodings, val_labels)

In [10]:

def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')  # use 'macro' if you want equal class weight
    return {'accuracy': acc, 'f1': f1}

In [11]:
from transformers import RobertaForSequenceClassification, RobertaConfig, Trainer, TrainingArguments, EarlyStoppingCallback
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

In [12]:

config = RobertaConfig.from_pretrained(
    'roberta-base',
    attention_probs_dropout_prob=0.2,
    hidden_dropout_prob=0.2,
    num_labels=3  # Change based on your sentiment classes
)

model = RobertaForSequenceClassification.from_pretrained('roberta-base', config=config)

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:

training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    learning_rate=2e-5,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    logging_dir='./logs',
)

In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: rubans0908 (rubans0908-avs-engineering-college) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.605900,0.518275,0.791849,0.792366
2,0.505200,0.530295,0.798763,0.797710


TrainOutput(global_step=2748, training_loss=0.581064768406675, metrics={'train_runtime': 1301.7421, 'train_samples_per_second': 33.776, 'train_steps_per_second': 2.111, 'total_flos': 2530624851707904.0, 'train_loss': 0.581064768406675, 'epoch': 2.0})

In [15]:
preds_output = trainer.predict(val_dataset)
preds = torch.argmax(torch.tensor(preds_output.predictions), axis=1)

print("Classification Report:\n")
print(classification_report(val_labels, preds, target_names=label_names))

Classification Report:

              precision    recall  f1-score   support

    negative       0.80      0.81      0.80      1572
     neutral       0.79      0.73      0.76      2236
    positive       0.81      0.87      0.84      1688

    accuracy                           0.80      5496
   macro avg       0.80      0.81      0.80      5496
weighted avg       0.80      0.80      0.80      5496



In [17]:
model.save_pretrained("sentiment-roberta-model")
tokenizer.save_pretrained("sentiment-roberta-model")


('sentiment-roberta-model/tokenizer_config.json',
 'sentiment-roberta-model/special_tokens_map.json',
 'sentiment-roberta-model/vocab.json',
 'sentiment-roberta-model/merges.txt',
 'sentiment-roberta-model/added_tokens.json')

In [20]:
from transformers import RobertaForSequenceClassification, RobertaTokenizer

model = RobertaForSequenceClassification.from_pretrained("/content/sentiment-roberta-model")
tokenizer = RobertaTokenizer.from_pretrained("/content/sentiment-roberta-model") # Use the correct path

In [44]:
user_text = input("Enter text to determine sentiment: ")

Enter text to determine sentiment: i don't love you 


In [45]:
inputs = tokenizer(user_text, return_tensors="pt", truncation=True, padding=True, max_length=128)

In [46]:
inputs = tokenizer(user_text, return_tensors="pt", truncation=True, padding=True, max_length=128)

In [47]:
with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.argmax(outputs.logits, dim=1)

In [48]:
predicted_sentiment_index = predictions.item()
predicted_sentiment = label_names[predicted_sentiment_index]
print(f"The predicted sentiment is: {predicted_sentiment}")

The predicted sentiment is: negative


In [49]:
eval_results = trainer.evaluate()

In [50]:
print(f"Validation Accuracy: {eval_results['eval_accuracy']:.4f}")

Validation Accuracy: 0.7988
